# Project Part 3

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/Matthew-Bustamante/CS39AA-Project-Cyberbullying/blob/main/project_part3.ipynb)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Matthew-Bustamante/CS39AA-Project-Cyberbullying/blob/main/project_part3.ipynb)


In [124]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn.functional as F
import torch.cuda
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [125]:
input_data_path = 'https://raw.githubusercontent.com/Matthew-Bustamante/CS39AA-Project-Cyberbullying/main/CyberBullying_Comments_Dataset.csv'
df = pd.read_csv(input_data_path)
df.head()

,Text,CB_Label
0,damn there is someones nana up here at beach w...,0
1,no kidding! dick clark was a corpse mechanical...,0
2,i read an article on jobros and thought damn w...,0
3,I got one fucking day of sprinkles and now it'...,0
4,I was already listening to Elliott smith and ...,0


In [126]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline, AutoModelForCausalLM
from datasets import Dataset, load_metric

In [127]:
#MODEL_NAME = "bert-base-cased"
MODEL_NAME = "gpt2"
MAX_LENGTH=50 

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
#model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, max_length=MAX_LENGTH, output_attentions=False, output_hidden_states=False)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, num_labels=3, max_length=MAX_LENGTH, output_attentions=False, output_hidden_states=False)
tokenizer.pad_token = tokenizer.eos_token

Creating Vocabulary for y(dependent variable).  Note that 0 = non-cyberbullying and 1 = cyberbullying

In [128]:
classes = df.CB_Label.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
class_idx2tok = dict((k, v) for k, v in enumerate(classes))
print(class_tok2idx)
print(class_idx2tok)

{0: 0, 1: 1}
{0: 0, 1: 1}


Creating a new column with these labels which will be the y that is used

In [129]:
df['label'] = df['CB_Label'].apply(lambda x: class_tok2idx[x])
df.head()

,Text,CB_Label,label
0,damn there is someones nana up here at beach w...,0,0
1,no kidding! dick clark was a corpse mechanical...,0,0
2,i read an article on jobros and thought damn w...,0,0
3,I got one fucking day of sprinkles and now it'...,0,0
4,I was already listening to Elliott smith and ...,0,0


In [130]:

sequence_0 = "Damn  hope you getz better soon !!!"
seq0_tokens = tokenizer(sequence_0, return_tensors="pt")
print(f"number of tokens in seq0 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)


number of tokens in seq0 is 10
{'input_ids': tensor([[43343,   220,  2911,   345,   651,    89,  1365,  2582,   220, 10185]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


tensor([[[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00],
         [3.1927e-10, 1.3907e-10, 7.4000e-11,  ..., 4.0189e-12,
          3.1584e-11, 3.7002e-12],
         [4.8246e-26, 1.6801e-26, 2.5647e-27,  ..., 4.5410e-27,
          3.6083e-27, 9.6959e-27],
         ...,
         [1.7994e-22, 2.6522e-23, 1.9690e-23,  ..., 2.6334e-24,
          3.4669e-25, 1.2228e-22],
         [9.0064e-22, 1.2230e-21, 8.9241e-22,  ..., 2.5080e-22,
          2.1852e-22, 2.0343e-20],
         [9.1879e-22, 9.9687e-22, 1.4542e-20,  ..., 4.2151e-23,
          2.6981e-23, 3.4057e-19]]], grad_fn=<SoftmaxBackward0>)

In [131]:
sequence_1 = "Shakespeare nerd!"
seq0_tokens = tokenizer(sequence_1, return_tensors="pt")
print(f"number of tokens in seq1 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)

number of tokens in seq1 is 4
{'input_ids': tensor([[ 2484, 20946, 34712,     0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


tensor([[[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00],
         [5.8101e-22, 6.5962e-22, 1.5302e-23,  ..., 1.0753e-21,
          2.1912e-21, 3.5497e-22],
         [3.2552e-15, 5.2959e-15, 6.4626e-17,  ..., 7.7948e-16,
          2.1389e-16, 2.3901e-16],
         [9.0530e-27, 2.5156e-27, 4.6379e-27,  ..., 3.5597e-28,
          4.5759e-28, 2.4334e-25]]], grad_fn=<SoftmaxBackward0>)

In [132]:
sequence_2 = "Is there ever a day that mattresses are not on sale?"
seq0_tokens = tokenizer(sequence_2, return_tensors="pt")
print(f"number of tokens in seq2 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)

number of tokens in seq2 is 13
{'input_ids': tensor([[ 3792,   612,  1683,   257,  1110,   326, 23963, 16746,   389,   407,
           319,  5466,    30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


tensor([[[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00],
         [3.3123e-25, 1.1844e-25, 1.3796e-26,  ..., 2.0306e-24,
          2.1939e-26, 2.2639e-25],
         [2.8305e-14, 2.3349e-14, 5.3661e-16,  ..., 1.9047e-14,
          2.7685e-14, 3.0814e-14],
         ...,
         [3.8406e-25, 4.2285e-26, 1.0269e-28,  ..., 3.8795e-27,
          5.6176e-27, 7.8150e-27],
         [2.6529e-39, 4.0599e-40, 1.3692e-41,  ..., 2.8487e-41,
          9.2864e-42, 1.3836e-40],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 4.2039e-45]]], grad_fn=<SoftmaxBackward0>)

In [133]:
sequence_3 = "Dood! You just justified your MBA. I pity the sorry ass of that downtrodden employee whose manager you'll soon become."
seq0_tokens = tokenizer(sequence_3, return_tensors="pt")
print(f"number of tokens in seq3 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)

number of tokens in seq3 is 27
{'input_ids': tensor([[   35,   702,     0,   921,   655, 14460,   534, 45517,    13,   314,
         26246,   262,  7926,   840,   286,   326,  8039,   305,  4742,  6538,
          3025,  4706,   345,  1183,  2582,  1716,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])}


tensor([[[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00],
         [1.7504e-23, 1.3703e-23, 1.4603e-24,  ..., 2.2641e-23,
          1.5879e-23, 3.6523e-23],
         [5.6825e-25, 9.9847e-26, 6.2765e-25,  ..., 4.8282e-26,
          8.1235e-26, 6.7659e-24],
         ...,
         [9.8091e-45, 1.1210e-44, 1.4013e-45,  ..., 0.0000e+00,
          8.4078e-45, 2.8026e-45],
         [2.8115e-26, 1.5521e-27, 6.8605e-29,  ..., 4.4561e-28,
          6.6813e-28, 4.5381e-28],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]]], grad_fn=<SoftmaxBackward0>)

In [134]:
ds_raw = Dataset.from_pandas(df[['label', 'Text']])

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [135]:
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)
ds = ds_raw.map(tokenize_function, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

In [136]:
ds[0]

{'label': 0,
 'Text': 'damn there is someones nana up here at beach with one. dont think ic an steal and get to you quickly tho ;(',
 'input_ids': [11043,
  77,
  612,
  318,
  617,
  1952,
  299,
  2271,
  510,
  994,
  379,
  10481,
  351,
  530,
  13,
  17666,
  892,
  14158,
  281,
  8711,
  290,
  651,
  284,
  345,
  2952,
  42796,
  2162,
  7,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [137]:
ds = ds.shuffle(seed=42)
ds[0]

{'label': 0,
 'Text': "haha it was great... i hate how the audio lags so badly though... it'd be more fun in real time lol",
 'input_ids': [71,
  12236,
  340,
  373,
  1049,
  986,
  1312,
  5465,
  703,
  262,
  6597,
  300,
  3775,
  523,
  11234,
  996,
  986,
  340,
  1549,
  307,
  517,
  1257,
  287,
  1103,
  640,
  19462,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [138]:
train_prop = 0.85
ds_train = ds.select(range(int(len(ds)*train_prop)))
ds_eval = ds.select(range(int(len(ds)*train_prop), len(ds)))

In [139]:
print(f"len(ds_train) = {len(ds_train)}")
print(f"len(ds_eval) = {len(ds_eval)}")

len(ds_train) = 9435
len(ds_eval) = 1665


In [140]:
import os 
os.environ["WANDB_DISABLED"] = "true"

In [141]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(num_train_epochs=10,
                                  do_train=True,
                                  report_to=None,
                                  output_dir="/kaggle/working",
                                  evaluation_strategy="steps",
                                  eval_steps=200,
                                  learning_rate=1e-5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32)

trainer = Trainer(model = model, 
                  args = training_args,
                  train_dataset = ds_train, 
                  eval_dataset = ds_eval,
                  compute_metrics = compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [142]:
if torch.cuda.is_available():
    device = "cuda:0"
    print("Using GPU")
else:
    device = "cpu"

Using GPU


In [143]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [144]:
torch.set_grad_enabled(True)
trainer.train()
trainer.evaluate()

ValueError: Expected input batch_size (1568) to match target batch_size (31).

In [ ]:
df_test = pd.read_csv("https://raw.githubusercontent.com/Matthew-Bustamante/CS39AA-Project-Cyberbullying/main/CyberBullying_Comments_Dataset.csv")
df_test.head()

In [ ]:
ds_test_raw = Dataset.from_pandas(df_test)
ds_test_raw[0]

In [ ]:
ds_test = ds_test_raw.map(tokenize_function, batched=True)

In [ ]:
preds = trainer.predict(test_dataset=ds_test)

In [ ]:
test_preds = np.apply_along_axis(np.argmax, 1, preds.predictions)

In [ ]:
df_test['preds'] = test_preds.tolist()
df_test['CB_Label'] = df_test['preds'].apply(lambda x: class_idx2tok[x])
df_test.head()